Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [1]:
import pandas as pd
import numpy as np

# Start coding here...

In [2]:
df = pd.read_csv("bank_marketing.csv")

for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

credit_default
--------------
credit_default
no         32588
unknown     8597
yes            3
Name: count, dtype: int64
mortgage
--------------
mortgage
yes        21576
no         18622
unknown      990
Name: count, dtype: int64
previous_outcome
--------------
previous_outcome
nonexistent    35563
failure         4252
success         1373
Name: count, dtype: int64
campaign_outcome
--------------
campaign_outcome
no     36548
yes     4640
Name: count, dtype: int64


In [3]:
df.columns

Index(['client_id', 'age', 'job', 'marital', 'education', 'credit_default',
       'mortgage', 'month', 'day', 'contact_duration', 'number_contacts',
       'previous_campaign_contacts', 'previous_outcome', 'cons_price_idx',
       'euribor_three_months', 'campaign_outcome'],
      dtype='object')

In [4]:
df.sample(8)

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome
1880,1880,38,blue-collar,divorced,basic.9y,no,no,may,24,70,4,0,nonexistent,93.994,4.855,no
30086,30086,34,unknown,married,basic.4y,no,no,apr,17,83,2,0,nonexistent,93.075,1.400,no
26604,26604,50,technician,married,high.school,no,yes,nov,19,75,2,1,failure,93.200,4.076,no
18807,18807,34,technician,divorced,unknown,no,no,jul,9,41,5,0,nonexistent,93.918,4.968,no
981,981,55,blue-collar,married,basic.4y,no,yes,may,1,1101,1,0,nonexistent,93.994,4.856,no
12623,12623,28,technician,divorced,university.degree,no,yes,jul,23,247,1,0,nonexistent,93.918,4.960,no
1158,1158,39,self-employed,married,high.school,no,yes,may,3,175,5,0,nonexistent,93.994,4.855,no
18784,18784,39,blue-collar,married,professional.course,no,yes,jul,1,55,15,0,nonexistent,93.918,4.968,no


In [5]:
client = df[
    ["client_id", "age", "job", "marital", "education", "credit_default", "mortgage"]
].copy()
client["job"] = client["job"].str.replace(".", "_")
client["education"] = client["education"].str.replace(".", "_")
client["education"] = client["education"].replace("unknown", np.nan)
client["credit_default"] = client["credit_default"] == "yes"
client["mortgage"] = client["mortgage"] == "yes"

client.head(3)

,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic_4y,False,False
1,1,57,services,married,high_school,False,False
2,2,37,services,married,high_school,False,True


In [6]:
campaign = df[
    [
        "client_id",
        "number_contacts",
        "contact_duration",
        "previous_campaign_contacts",
        "previous_outcome",
        "campaign_outcome",
        "day",
        "month",
    ]
].copy()

campaign["year"] = 2022

campaign["previous_outcome"] = campaign["previous_outcome"] == "success"
campaign["campaign_outcome"] = campaign["campaign_outcome"] == "yes"

campaign["last_contact_date"] = pd.to_datetime(
    campaign["day"].astype(str)
    + " "
    + campaign["month"]
    + " "
    + campaign["year"].astype(str),
    format="%d %b %Y",
)

campaign.drop(columns=["day", "month", "year"], inplace=True)

campaign.sample(3)

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
35491,35491,1,55,0,False,False,2022-05-10
3264,3264,1,500,0,False,True,2022-05-15
14778,14778,4,145,0,False,False,2022-07-17


In [7]:
economics = df[["client_id", "cons_price_idx", "euribor_three_months"]].copy()

In [8]:
client.to_csv("client.csv", index=None)
campaign.to_csv("campaign.csv", index=None)
economics.to_csv("economics.csv", index=None)